In [1]:
import yaml
from transformer_io.dataset import GeneIdTokenizer
from datamodules import MappedCollectionDataModule
from pathlib import Path
import pandas as pd

💡 connected lamindb: mojtababahrami/lamin-storage


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_root = Path('/Users/mojtaba.bahrami/Development/transformer-io-data/')
dataset_path = data_root / 'data/'

In [4]:
# import subprocess
# for filename in ["52c749a1-1c68-4304-883d-f122fec17603.h5ad", 'ce6a5773-c83a-4ade-be28-bb6aa083080a.h5ad']:
#     bash_command = f"curl -o {data_root}/{filename} 'https://datasets.cellxgene.cziscience.com/{filename}'"
#     subprocess.run(bash_command, shell=True)
# ! scp hpc-build01:/lustre/groups/ml01/projects/transformer-io/gene_token_mapping.pkl $data_root
# ! scp hpc-build01:/lustre/groups/ml01/projects/transformer-io/protein_token_mapping.pkl $data_root
# ! scp hpc-build01:/lustre/groups/ml01/projects/transformer-io/chromatin_token_mapping.pkl $data_root

In [13]:
gene_mapping_path = data_root / 'gene_token_mapping.pkl'
gene_mapping = pd.read_pickle(gene_mapping_path).to_dict()

split = {'train': ['52c749a1-1c68-4304-883d-f122fec17603.h5ad', 
                   'ce6a5773-c83a-4ade-be28-bb6aa083080a.h5ad'
                   ],
          'val':  ['52c749a1-1c68-4304-883d-f122fec17603.h5ad', 
                   'ce6a5773-c83a-4ade-be28-bb6aa083080a.h5ad']
          }

# yaml_file_path = "/home/icb/mojtaba.bahrami/projects/contrastive-transformer/conf/split/split_v1_large.yaml"
# with open(yaml_file_path, "r") as file:
#     split = yaml.safe_load(file)


dataset_args = {'train':{'n_tokens': 200, 'sub_sample_frac': 1.0}, 'val':{'n_tokens': 200, 'sub_sample_frac': 1.0}}
dataloader_args = {
  'train':
    {'within_dataset_sampling': True,
    'shuffle': True,
    'num_workers': 1,
    'drop_last': True,
    'batch_size': 32,
    },
  'val':
    {'within_dataset_sampling': False,
    'shuffle': True,
    'num_workers': 1,
    'drop_last': True,
    'batch_size': 32,
    },
}

datamodule_args = {
    'dataset_path': dataset_path,
    'split': split,
    'columns': [],
    'normalization': 'log1p',
    'dataset_kwargs': dataset_args,
    'dataloader_kwargs': dataloader_args,
    'tokenizer': GeneIdTokenizer(gene_mapping)
}

datamodule = MappedCollectionDataModule(**datamodule_args)

Dataset 0: 12934 / 15333 tokens
Dataset 1: 13730 / 16756 tokens
coverage macro: 0.8314740251107517
covarage micro: 0.8334991815514783
Dataset 0: 12934 / 15333 tokens
Dataset 1: 13730 / 16756 tokens
coverage macro: 0.8314740251107517
covarage micro: 0.8334991815514783


In [14]:
train_loader = datamodule.train_dataloader() 
for i, batch in enumerate(train_loader):
    break

Creating train dataloader by 466 batches of size 32 over 14925 samples; num_replicas=1; sum of indices: 57250200
💡 connected lamindb: mojtababahrami/lamin-storage


In [15]:
batch

{'tokens': tensor([[19038, 17432,  6778,  ...,  3895,  8883, 15323],
         [17260, 11712,  5986,  ..., 13571, 16432, 12058],
         [ 5885, 16486, 14126,  ..., 11607, 14446,  4644],
         ...,
         [ 7603,  9783, 16375,  ...,  4972, 17312, 11975],
         [11890,  5687, 10062,  ..., 10015, 15104,  9366],
         [11924,  2899,  2996,  ...,  5479, 11181, 18241]]),
 'values': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.6931,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 'dataset_id': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1])}

In [16]:
val_loader = datamodule.val_dataloader() 
for i, batch in enumerate(val_loader):
    break

Creating val dataloader by 466 batches of size 32 over 14925 samples; num_replicas=1; sum of indices: 57250200
💡 connected lamindb: mojtababahrami/lamin-storage


In [17]:
batch

{'tokens': tensor([[13887,   477,  5883,  ...,  3121,  4240, 14747],
         [ 9683,  3272, 15648,  ...,  1489, 14902, 12859],
         [15838, 13224,   350,  ..., 10608,  5855, 18993],
         ...,
         [ 2452,  5802,  5883,  ...,  1332, 11996,  8695],
         [13297,  1563,  9568,  ..., 14363, 18882, 15164],
         [ 6665, 16739, 15511,  ..., 18151,   606,  2321]]),
 'values': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.0986, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.6931, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 'dataset_id': tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 1, 0, 1, 1, 1, 1, 0])}